In [ ]:
import os
import cv2
import numpy as np
import moviepy.video.io.ImageSequenceClip
from moviepy.editor import VideoFileClip
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM

time = 31
channels = 33

for codec in ['libx264']:
    out_folder = f'outputs/moviepy/{codec}'
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
    for ci in range(channels):
        vid_imgs = []
        for ti in range(time):
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            vid_imgs.append(im)
        clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(vid_imgs, fps=24)
        clip.write_videofile(f"{out_folder}/{ci+1}.mp4", audio=False, preset='veryslow', logger=None, codec=codec, bitrate='200k')
    psnrs = []
    ssims = []
    for ci in range(channels):
        clip = VideoFileClip(f"{out_folder}/{ci+1}.mp4")
        for ti, frames in enumerate(clip.iter_frames()):
            if ti >= time:
                continue
            gray_frames = cv2.cvtColor(frames, cv2.COLOR_RGB2GRAY)
            im = cv2.imread(f'dataset/hyperspectral-video-33bands/f{ti+1}/MSV_1_{ci+1}.png')
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            psnrs.append(PSNR(gray_frames, im))
            ssims.append(SSIM(gray_frames, im))
    with open(f"{out_folder}/metrics.txt",'w') as f:
        f.write(f"MEAN PSNR:{np.array(psnrs).mean()}\n")
        f.write(f"MEAN SSIM:{np.array(ssims).mean()}\n")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

mean_psnrs1 = [31.96, 35.20, 38.58, 39.96, 40.44]
mean_ssims1 = [0.87, 0.91, 0.94, 0.95, 0.95]
compressed_ratio1 = [100 * (1 - 275175 / 180338675),
                     100 * (1 - 438066 / 180338675),
                     100 * (1 - 1479375 / 180338675),
                     100 * (1 - 2279356 / 180338675),
                     100 * (1 - 3560501 / 180338675),
                    ]

mean_psnrs2 = [33.86, 36.95, 39.60]
compressed_ratio2 = [100 * (1 - 830663 / 180338675),
                     100 * (1 - 3230919 / 180338675),
                     100 * (1 - 12750023 / 180338675),
                    ]

sns.set()
fig, ax2 = plt.subplots(1, 1, facecolor='w')
ax = ax2.twinx()
ax.plot(compressed_ratio1, mean_psnrs1, '-', linewidth=2.5, color="maroon", label="Video Encoding (PSNR)")
ax2.plot(compressed_ratio1, mean_ssims1, '--', linewidth=2.5, color="maroon", label="Video Encoding (SSIM)")
# ax.plot(compressed_ratio2, mean_psnrs2, linewidth=2.5, color="lightcoral", label="SIREN")
ax.set_ylim(top=41.8)
ax2.set_ylim(top=0.97)
ax.set_ylabel("Mean PSNR")
ax2.set_ylabel("Mean SSIM")
ax.set_xlabel(r"Compression Rate")
handles, labels = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles += handles2
labels += labels2
ax.legend(handles, labels, loc='upper right', ncol=1, prop={'size': 11})
fig.tight_layout()
plt.savefig('outputs/baseline.pdf')